# Colab Initialization

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/radistoubalidis/JSRepair.git

!python -m pip install lightning
!pip install datasets
!pip install python-dotenv
!pip install rouge-score
!pip install diff-match-patch
!pip install gspread google-auth

Cloning into 'JSRepair'...
remote: Enumerating objects: 552, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 552 (delta 45), reused 34 (delta 16), pack-reused 484 (from 1)
Receiving objects: 100% (552/552), 2.15 MiB | 2.31 MiB/s, done.
Resolving deltas: 100% (377/377), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━

In [3]:
%cd ./JSRepair

/content/JSRepair


# Dependencies

In [4]:
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
from typing import Any
from sklearn.model_selection import train_test_split
from modules.filters import add_labels, count_comment_lines
import os
import pandas as pd
import sqlite3
from modules.TrainConfig import init_logger, init_checkpoint, Trainer
from modules.datasets import CodeT5Dataset
from diff_match_patch import diff_match_patch
from transformers import RobertaTokenizer
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader
from transformers import default_data_collator
from modules.models import CodeT5
from lightning import Trainer as plTrainer

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

In [5]:
HF_DIR = 'Salesforce/codet5-base'
TOKENIZER_MAX_LENGTH = 512 #int(input('Tokenizer Max length: '))
DB_PATH = 'commitpack-datasets.db' if os.path.exists('commitpack-datasets.db') else '/content/drive/MyDrive/Thesis/commitpack-datasets.db'
DB_TABLE = 'commitpackft_classified_train'
if not os.path.exists(DB_PATH):
    raise RuntimeError('sqlite3 path doesnt exist.')
VAL_SIZE = 0.3
LOG_PATH = 'logs' if os.path.exists('logs') else '/content/drive/MyDrive/Thesis/logs'
VERSION = int(input('Training version: '))
LOAD_FROM_CPKT = input("Load from existing model (type cpkt path if true): ")
DEBUG = True if int(input('Debug Run (1,0): ')) == 1 else False
BATCH_SIZE = 8 if DEBUG is True else 64
CPKT_PATH = 'checkpoints' if os.path.exists('checkpoints') else '/content/drive/MyDrive/Thesis/checkpoints'
DROPOUT_RATE = float(input('Type dropout rate for classifier: '))
NUM_EPOCHS = int(input('Type number of train epochs: '))
WITH_LAYER_NORM = True
WITH_ACTIVATION = True
LEARNING_RATE = float(input("Type initial lr,(default = 1e-3): "))
BIMODAL_TRAIN = True if int(input('Combine commit messages with codes (1,0): ')) == 1 else False
NEW_CPKT = True if int(input('Save to new checkpoint path (1,0): ')) == 1 else False

classLabels = {
        "functionality" : 0.,
        "ui-ux" : 0.,
        "compatibility-performance" : 0.,
        "network-security" : 0.,
        "general": 0.
    }

num_classes = len(classLabels.keys())
modelSize = HF_DIR.split('-')[-1]
MODEL_DIR = f"CodeT5_{modelSize}_JS_{num_classes}classes_{TOKENIZER_MAX_LENGTH}MaxL"
con = sqlite3.connect(DB_PATH)

Training version: 906
Load from existing model (type cpkt path if true): /content/drive/MyDrive/Thesis/checkpoints/CodeT5_base_JS_5classes_512MaxL_v906-v1.ckpt
Debug Run (1,0): 0
Type dropout rate for classifier: 0.5
Type number of train epochs: 5
Type initial lr,(default = 1e-3): 1e-2
Combine commit messages with codes (1,0): 0
Save to new checkpoint path (1,0): 0


# Training

## Create Classification Labels

```json
{
    "mobile" : 0,
    "functionality" : 0,
    "ui-ux" : 0,
    "compatibility-performance" : 0,
    "network-security" : 0,
    "general": 0
}

Ένα δείγμα που κατηγοριοποιήθηκε ως σφάλμα λειτουργικότητας(functionality) και ui-ux θα έχει διάνυσμα ταξινόμησης ->
[0,1,1,0,0,0]
```


In [6]:
ds_df = pd.read_sql_query("select * from commitpackft_classified_train", con)
ds_df['class_labels'] = ds_df['bug_type'].apply(lambda bT: add_labels(bT.split(','), classLabels))

ds_df = ds_df[ds_df['bug_type'] != 'mobile']
ds_df = ds_df[ds_df['old_contents'].str.len() > 0]

if DEBUG:
    ds_df = ds_df.sample(20)

## Data Preprocess

### Filter out outliers

In [7]:
ds_df['old_contents_comment_lines_count'] = ds_df['old_contents'].apply(lambda sample: count_comment_lines(sample))
ds_df['new_contents_comment_lines_count'] = ds_df['new_contents'].apply(lambda sample: count_comment_lines(sample))

# Filter out samples where the sum of comment lines increased more than 3 lines
# to prevent excessive masking
ds_df = ds_df[abs(ds_df['old_contents_comment_lines_count'] - ds_df['new_contents_comment_lines_count']) <= 3]
# Filter out samples with more than 10 comment lines
ds_df = ds_df[(ds_df['old_contents_comment_lines_count'] < 10) & (ds_df['new_contents_comment_lines_count'] < 10)]

dmp = diff_match_patch()

def compute_diffs(sample: dict):
    # Compute the differences
    diffs = dmp.diff_main(sample['old_contents'], sample['new_contents'])
    dmp.diff_cleanupSemantic(diffs)
    # Count the changes
    return sum(1 for diff in diffs if diff[0] == 1)  # 0 means no change

ds_df['num_changes'] = ds_df.apply(lambda sample: compute_diffs(sample), axis=1)

# Filter out samples with more than 3 changes in the code
ds_df = ds_df[ds_df['num_changes'] <= 3]

# filter out samples with more than 50 lines of code
ds_df = ds_df[(ds_df['old_contents'].str.count('\n') <= 50) & (ds_df['new_contents'].str.count('\n') <= 50)]

# filter out samples with more than 2 bug types
ds_df = ds_df[ds_df['bug_type'].str.count(',') < 2]
print(f"Total training samples after filtering: {len(ds_df)}")

### Concatenate Commit Message with the old contents
- This way, the commit message is directly provided as additional context, and the models (T5, Bert) can process both the buggy code and the commit message in a unified manner.
- This approach will allow the model to learn the relationship between the commit message and the changes made to the code.

In [8]:
tokenizer = RobertaTokenizer.from_pretrained(HF_DIR)
old_codes = ds_df[['message', 'old_contents', 'class_labels']]
new_codes = ds_df[['message', 'new_contents', 'class_labels']]
if BIMODAL_TRAIN:
    old_codes['input_seq'] = '/* ' + old_codes['message'] + '*/\n' + tokenizer.sep_token + '\n' + old_codes['old_contents']
    new_codes['output_seq'] = '/* ' + new_codes['message'] + '*/\n' + tokenizer.sep_token + '\n' + new_codes['new_contents']
else:
    old_codes['input_seq'] = old_codes['old_contents'].copy()
    new_codes['output_seq'] = new_codes['new_contents'].copy()

TRAIN_old, VAL_old, TRAIN_new, VAL_new = train_test_split(old_codes, new_codes, test_size=VAL_SIZE, random_state=42)

print(f"Total training samples: {len(TRAIN_old)}")
print(f"Total validation samples: {len(VAL_old)}")

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

Total training samples: 4646
Total validation samples: 1992


<ipython-input-8-23b72b160dc9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_codes['input_seq'] = old_codes['old_contents'].copy()
<ipython-input-8-23b72b160dc9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_codes['output_seq'] = new_codes['new_contents'].copy()


## Prepare Model Input

In [9]:
TOKENIZER_MAX_LENGTH = 512

TRAIN_encodings = tokenizer(
    TRAIN_old['input_seq'].tolist(),
    max_length=TOKENIZER_MAX_LENGTH,
    pad_to_max_length=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True
)

VAL_encodings = tokenizer(
    VAL_old['input_seq'].tolist(),
    max_length=TOKENIZER_MAX_LENGTH,
    pad_to_max_length=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True
)

TRAIN_decodings = tokenizer(
    TRAIN_new['output_seq'].tolist(),
    max_length=TOKENIZER_MAX_LENGTH,
    pad_to_max_length=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True
)

VAL_decodings = tokenizer(
    VAL_new['output_seq'].tolist(),
    max_length=TOKENIZER_MAX_LENGTH,
    pad_to_max_length=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True
)

TRAIN_classes = torch.tensor(TRAIN_old['class_labels'].tolist())
VAL_classes = torch.tensor(VAL_old['class_labels'].tolist())



TRAIN_dataset = CodeT5Dataset(TRAIN_encodings, TRAIN_decodings, TRAIN_classes)
VAL_dataset = CodeT5Dataset(VAL_encodings, VAL_decodings, VAL_classes)


# Class weights
# pos_weight[i] = (Number of negative samples for class i) / (Number of positive samples for class i)
num_samples = TRAIN_classes.size(0)
num_classes = TRAIN_classes.size(1)

pos_counts = torch.sum(TRAIN_classes, dim=0)
neg_counts = num_samples - pos_counts
class_weights = neg_counts / (pos_counts + 1e-6)
class_weights = class_weights.numpy()

## Initialize Training Settings

In [10]:
logger = init_logger(log_path=LOG_PATH, model_dir=MODEL_DIR, version=VERSION)
checkpoint = init_checkpoint(cpkt_path=CPKT_PATH, model_dir=MODEL_DIR, version=VERSION, targetMetric='val_auxilary_loss')
trainer = Trainer(checkpoint,logger,debug=DEBUG, num_epochs=NUM_EPOCHS, precision='32-true')

if len(LOAD_FROM_CPKT) > 0 and os.path.exists(LOAD_FROM_CPKT):
    model = CodeT5.load_from_checkpoint(
        LOAD_FROM_CPKT,
        class_weights=class_weights,
        model_dir=HF_DIR,
        num_classes=num_classes,
        dropout_rate=DROPOUT_RATE,
        with_activation=True,
        with_layer_norm=True,
        lr=LEARNING_RATE
    )
else:
    model = CodeT5(
        class_weights=class_weights,
        model_dir=HF_DIR,
        num_classes=num_classes,
        dropout_rate=DROPOUT_RATE,
        with_activation=True,
        with_layer_norm=True,
        lr=LEARNING_RATE
    )

model.model.train()
model.classifier.train()

if DEBUG:
    model.to('cpu')

dataloader = DataLoader(TRAIN_dataset, batch_size=BATCH_SIZE,num_workers=14, shuffle=True, collate_fn=default_data_collator)
val_dataloader = DataLoader(VAL_dataset, batch_size=BATCH_SIZE, num_workers=14, collate_fn=default_data_collator)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 14 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## Run Training

In [11]:
if len(LOAD_FROM_CPKT) > 0 and os.path.exists(LOAD_FROM_CPKT) and not NEW_CPKT:
    trainer.fit(
        model,
        train_dataloaders=dataloader,
        val_dataloaders=val_dataloader,
        ckpt_path=LOAD_FROM_CPKT
    )
else:
    trainer.fit(
        model,
        train_dataloaders=dataloader,
        val_dataloaders=val_dataloader
    )

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /content/drive/MyDrive/Thesis/checkpoints exists and is not empty.
INFO: Restoring states from the checkpoint pat

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Training: |          | 0/? [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 3, global step 292: 'val_auxilary_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 3, global step 292: 'val_auxilary_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 4, global step 365: 'val_auxilary_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 4, global step 365: 'val_auxilary_loss' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


## Save Model Config to CSV

In [12]:
if not DEBUG:
    import gspread
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    from google.auth import default

    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    spreadsheet = gc.open("model-configs").sheet1
    modelConfig = {
            'name': MODEL_DIR,
            'tokenizer_max_length': TOKENIZER_MAX_LENGTH,
            'num_classes': num_classes,
            'dropout_rate': DROPOUT_RATE,
            'with_activation': WITH_ACTIVATION,
            'with_layer_norm': WITH_LAYER_NORM,
            'learning_rate': LEARNING_RATE,
            'bimodal_train': BIMODAL_TRAIN
    }
    spreadsheet.append_row(list(modelConfig.values()))